In [1]:
import pandas as pd
import geopy.distance
import numpy as np
import sys
sys.path.insert(0, '/Users/ewd/ShrinkSmart/')
from ShrinkSmart import ShrinkSmart

scalarDF = pd.read_csv("scalar-metrics.csv").drop(["Unnamed: 0"], axis=1)

services = pd.read_csv("connection-metrics-services.csv").drop(["Unnamed: 0"], axis=1)
labels = list(services["label"].unique())
cities = list(scalarDF["City"].unique())
distanceLevels = [5, 25, 50, 100]
densityDF = pd.DataFrame(columns=["City"] + distanceLevels + ["Label"])
for label in labels:
    for city in cities:
        pop = float(scalarDF[scalarDF["City"] == city]["Population"])
        distList = []
        for distance in distanceLevels:
            distList.append(len(services[services["label"] == label][services["City"] == city][services["rating"] > 3][services["km distance"] <= distance]))
        distList = [city] + distList + [label]
        cityDF = pd.DataFrame([distList], columns=["City"] + distanceLevels + ["Label"])
        densityDF = densityDF.append(cityDF)
        
densityDF = densityDF.reset_index(drop=True)

densityDF

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,City,5,25,50,100,Label
0,"Altoona, IA",0,6,7,7,Pay-Day Loans
1,"Anita, IA",0,0,0,0,Pay-Day Loans
2,"Audubon, IA",0,0,0,0,Pay-Day Loans
3,"Buffalo Center, IA",0,0,0,0,Pay-Day Loans
4,"Columbus Junction, IA",0,0,0,0,Pay-Day Loans
5,"Corning, IA",0,0,0,0,Pay-Day Loans
6,"Correctionville, IA",0,0,0,0,Pay-Day Loans
7,"Elma, IA",0,0,0,0,Pay-Day Loans
8,"Epworth, IA",0,0,0,3,Pay-Day Loans
9,"Everly, IA",0,0,0,0,Pay-Day Loans


In [38]:
import re, string
pattern = re.compile('[\W_]+')
pattern.sub('', string.printable)

labels = list(densityDF["Label"].unique())
newLabels = [pattern.sub('', x.title()) for x in labels]
labelsDict = dict(zip(labels,newLabels))
labelsDict

{'Pay-Day Loans': 'PayDayLoans',
 'groceries': 'Groceries',
 'Pawn Shops': 'PawnShops',
 'convenience stores': 'ConvenienceStores',
 'restaurants': 'Restaurants',
 'gas stations': 'GasStations',
 'optometrist': 'Optometrist',
 'dollar store': 'DollarStore',
 'child care': 'ChildCare',
 'pharmacy': 'Pharmacy',
 'dental': 'Dental',
 'Check Cashing': 'CheckCashing'}

In [39]:
outDF = pd.DataFrame()
outDF["City"] = ShrinkSmart.cities

def labelTransfrom(row):
    city = row["City"]
    

for label in labels:
    label5 = labelsDict[label] + "5"
    label25 = labelsDict[label] + "25"
    label50 = labelsDict[label] + "50"
    label100 = labelsDict[label] + "100"        
    outDF[label5] = outDF[label25] = outDF[label50] = outDF[label100] = 0
    
for city in list(scalarDF["City"].unique()):
    for label in labels:
        label5 = labelsDict[label] + "5"
        label25 = labelsDict[label] + "25"
        label50 = labelsDict[label] + "50"
        label100 = labelsDict[label] + "100"     
        value = float((densityDF[densityDF["City"] == city][densityDF["Label"] == label])[5])
        outDF.loc[outDF["City"] == city, [label5]] = value
        value = float((densityDF[densityDF["City"] == city][densityDF["Label"] == label])[25])
        outDF.loc[outDF["City"] == city, [label25]] = value
        value = float((densityDF[densityDF["City"] == city][densityDF["Label"] == label])[50])
        outDF.loc[outDF["City"] == city, [label50]] = value
        value = float((densityDF[densityDF["City"] == city][densityDF["Label"] == label])[100])
        outDF.loc[outDF["City"] == city, [label100]] = value        

outDF

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,City,PayDayLoans5,PayDayLoans25,PayDayLoans50,PayDayLoans100,Groceries5,Groceries25,Groceries50,Groceries100,PawnShops5,...,Pharmacy50,Pharmacy100,Dental5,Dental25,Dental50,Dental100,CheckCashing5,CheckCashing25,CheckCashing50,CheckCashing100
0,"Altoona, IA",0.0,6.0,7.0,7.0,3.0,7.0,7.0,7.0,0.0,...,14.0,16.0,1.0,7.0,7.0,7.0,0.0,0.0,0.0,0.0
1,"Lake Park, IA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Williamsburg, IA",0.0,0.0,5.0,6.0,0.0,1.0,17.0,17.0,0.0,...,9.0,12.0,1.0,1.0,9.0,9.0,0.0,0.0,0.0,0.0
3,"Batavia, IA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Buffalo Center, IA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"Epworth, IA",0.0,0.0,0.0,3.0,0.0,7.0,15.0,17.0,0.0,...,6.0,12.0,0.0,10.0,10.0,17.0,0.0,0.0,0.0,0.0
6,"Le Claire, IA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"St. Charles, IA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"Fruitland, IA",0.0,0.0,2.0,6.0,0.0,1.0,1.0,1.0,0.0,...,6.0,12.0,0.0,3.0,7.0,18.0,0.0,0.0,0.0,2.0
9,"Le Mars, IA",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
outDF.to_csv("services-scalars.csv")

In [41]:
edge = dict()

label = "Pay-Day Loans"

frames = dict()

for label in services["label"].unique():

    for city in services["City"].unique():
        edge[city] = dict()
        total = services[services["City"] == city][services["label"] == label]["num reviews"].sum()
        for dest in services[services["City"] == city][services["label"] == label]["destination"].unique():
            idx = dest
            weight = services[services["City"] == city][services["label"] == label][services["destination"] == dest]["num reviews"].sum()
            edge[city][idx] = float(weight)/float(total)

    connectionDF = pd.DataFrame(edge).T.fillna("0")
    frames[label] = connectionDF
    
pd.concat(frames.values(), keys=frames.keys()).reorder_levels([1,0])


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,,"Adair, IA","Adel, IA","Adrian, MN","Afton, IA","Agency, IA","Ainsworth, IA","Aledo, IL","Allendale, MO","Alton, IA","Altoona, IA",...,"Cedar Rapids, IA","Everly, IA","Grand Meadow, MN","Marengo, IA","Milford, IA","Montezuma, IA","Ottumwa, IA","Perry, IA","Royal, IA","Viola, IL"
"Altoona, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Williamsburg, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Batavia, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Epworth, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Fruitland, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Correctionville, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Elma, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Hamburg, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Everly, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Sheffield, IA",Pay-Day Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
pd.concat(frames.values(), keys=frames.keys()).reorder_levels([1,0]).fillna(0).to_csv("service_matrix.csv")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
